In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def bbox_overlaps_giou(bboxes1, bboxes2):
    """Calculate the gious between each bbox of bboxes1 and bboxes2.
    Args:
        bboxes1(ndarray): shape (n, 4)
        bboxes2(ndarray): shape (k, 4)
    Returns:
        gious(ndarray): shape (n, k)
    """


    #bboxes1 = torch.FloatTensor(bboxes1)
    #bboxes2 = torch.FloatTensor(bboxes2)
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = torch.zeros((cols, rows))
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0]) * (
        bboxes1[:, 3] - bboxes1[:, 1])
    area2 = (bboxes2[:, 2] - bboxes2[:, 0]) * (
        bboxes2[:, 3] - bboxes2[:, 1])

    inter_max_xy = torch.min(bboxes1[:, 2:],bboxes2[:, 2:])

    inter_min_xy = torch.max(bboxes1[:, :2],bboxes2[:, :2])

    out_max_xy = torch.max(bboxes1[:, 2:],bboxes2[:, 2:])

    out_min_xy = torch.min(bboxes1[:, :2],bboxes2[:, :2])

    inter = torch.clamp((inter_max_xy - inter_min_xy), min=0)
    inter_area = inter[:, 0] * inter[:, 1]
    outer = torch.clamp((out_max_xy - out_min_xy), min=0)
    outer_area = outer[:, 0] * outer[:, 1]
    union = area1+area2-inter_area
    closure = outer_area

    ious = inter_area / union - (closure - union) / closure
    ious = torch.clamp(ious,min=-1.0,max = 1.0)
    #print(ious)
    if exchange:
        ious = ious.T
    return ious

class GiouLoss(nn.Module):
    """
        This criterion is a implemenation of Giou Loss, which is proposed in 
        Generalized Intersection over Union Loss for: A Metric and A Loss for Bounding Box Regression.
            Loss(loc_p, loc_t) = 1-GIoU
        The losses are summed across observations for each minibatch.
        Args:
            size_sum(bool): By default, the losses are summed over observations for each minibatch.
                                However, if the field size_sum is set to False, the losses are
                                instead averaged for each minibatch.
            predmodel(Corner,Center): By default, the loc_p is the Corner shape like (x1,y1,x2,y2)
            The shape is [num_prior,4],and it's (x_1,y_1,x_2,y_2)
            loc_p: the predict of loc
            loc_t: the truth of boxes, it's (x_1,y_1,x_2,y_2)
            
    """
    def __init__(self,pred_mode = 'Center',size_sum=True,variances=None):
        super(GiouLoss, self).__init__()
        self.size_sum = size_sum
        self.pred_mode = pred_mode
        self.variances = variances
    def forward(self, loc_p, loc_t,prior_data):
        num = loc_p.shape[0] 
        
        if self.pred_mode == 'Center':
            decoded_boxes = decode(loc_p, prior_data, self.variances)
        else:
            decoded_boxes = loc_p
        #loss = torch.tensor([1.0])
        gious =1.0 - bbox_overlaps_giou(decoded_boxes,loc_t)
        
        loss = torch.sum(gious)
     
        if self.size_sum:
            loss = loss
        else:
            loss = loss/num
        return 5*loss

In [26]:
n, k, = 2, 2
bboxes1=torch.rand(n, 4)*100
bboxes2=torch.rand(k, 4)*100
print(bboxes1, bboxes2)
print(bboxes1[:, 2:].shape, bboxes2[:, 2:].shape)
torch.min(bboxes1[:, 2:],bboxes2[:, 2:])

tensor([[84.6549, 92.1703, 48.7778, 26.3684],
        [71.1937, 22.3946, 99.2003, 36.0609]]) tensor([[56.7005, 68.4486, 97.0677, 28.8729],
        [50.4809, 27.5771, 18.0142, 72.6310]])
torch.Size([2, 2]) torch.Size([2, 2])


tensor([[48.7778, 26.3684],
        [18.0142, 36.0609]])